# Day 1: The Tyranny of the Rocket Equation
https://adventofcode.com/2019/day/1

In [ ]:
import numpy as np
import urllib.request
import math

## Part 1

In [ ]:
def calculateFuel(modules, debug=False):
    if debug:
        output = [(x // 3) - 2 for x in modules]
        print('input', modules)
        print('output', output)
        return np.sum(output)
    else:
        return np.sum([ (x // 3) - 2 for x in modules])

def calculateFuelByModule(modules):
    return [ (x // 3) - 2 for x in modules]

In [ ]:
input = np.random.randint(1, 5000, 5)
input

In [ ]:
calculateFuel(input, debug=True)

In [ ]:
calculateFuelByModule(input)

### Tests
* For a mass of 12, divide by 3 and round down to get 4, then subtract 2 to get 2.
* For a mass of 14, dividing by 3 and rounding down still yields 4, so the fuel required is also 2.
* For a mass of 1969, the fuel required is 654.
* For a mass of 100756, the fuel required is 33583.

In [ ]:
massTest = [ 12, 14, 1969, 100756 ]
calculateFuel(massTest, debug=True)

In [ ]:
print(calculateFuel([12], debug = True))
print(calculateFuel([14], debug = True))
print(calculateFuel([1969], debug = True))
print(calculateFuel([100756], debug = True))

In [ ]:
print(calculateFuelByModule([12]))
print(calculateFuelByModule([14]))
print(calculateFuelByModule([1969]))
print(calculateFuelByModule([100756]))

### Solution

In [ ]:
input_1 = r'C:\Users\rodrigo\Notebooks\aoc2019-input-day1.txt'
with open(input_1, 'r') as f:
    data = [int(data) for data in f.read().split('\n') if len(data) > 0]

In [ ]:
calculateFuel(data, debug=True)

## Part 2

### Tests
* A module of mass 14 requires 2 fuel. This fuel requires no further fuel (2 divided by 3 and rounded down is 0, which would call for a negative fuel), so the total fuel required is still just 2.
* At first, a module of mass 1969 requires 654 fuel. Then, this fuel requires 216 more fuel (654 / 3 - 2). 216 then requires 70 more fuel, which requires 21 fuel, which requires 5 fuel, which requires no further fuel. So, the total fuel required for a module of mass 1969 is 654 + 216 + 70 + 21 + 5 = 966.
* The fuel required by a module of mass 100756 and its fuel is: 33583 + 11192 + 3728 + 1240 + 411 + 135 + 43 + 12 + 2 = 50346.

Nótese que se calcula el fuel de la masa igual que el del fuel adicional.

In [ ]:
def calculateAdditionalFuel(fuelSoFar):
#     print('Entrando', fuelSoFar)
    extra_fuel = calculateFuelByModule([fuelSoFar])[0]
    if extra_fuel <= 0:
        return 0
    
    return extra_fuel + calculateAdditionalFuel(extra_fuel)

In [ ]:
mass = 14
initial_fuel = calculateFuelByModule([mass])[0]
final_fuel = initial_fuel + calculateAdditionalFuel(initial_fuel)
print(final_fuel)

In [ ]:
calculateAdditionalFuel(14)

In [ ]:
mass = 1969
initial_fuel = calculateFuelByModule([mass])[0]
final_fuel = initial_fuel + calculateAdditionalFuel(initial_fuel)
print(final_fuel)

In [ ]:
calculateAdditionalFuel(1969)

In [ ]:
mass = 100756
initial_fuel = calculateFuelByModule([mass])[0]
final_fuel = initial_fuel + calculateAdditionalFuel(initial_fuel)
print(final_fuel)

In [ ]:
calculateAdditionalFuel(100756)

In [ ]:
# [(m, calculateFuelByModule([m])[0], calculateAdditionalFuel(calculateFuelByModule([m])[0]), calculateAdditionalFuel(m)) for m in data]

In [ ]:
np.sum([calculateAdditionalFuel(m) for m in data])

# Day 2: 1202 Program Alarm
https://adventofcode.com/2019/day/2

## Part 1

In [ ]:
class Operation():
    endComputations = False
    #TODO: Podríamos poner aquí el "tamaño" de la operación (los parámetros 
    # que tiene) para así avanzar el contador de programa. Y para validar en el programa
    # que están todos los parámetros necesarios. Para casos donde haya
    # operaciones con un número de parámetros diferente.
    
    def mustEndComputations(self):
        return self.endComputations
    
    def checkParameter(self, program, PCparam):
        if program[PCparam] > len(program):
            raise Exception('Parameter at {} out of bounds {}'.format(program[PCparam], len(program)))
    
    def checkValidation(self, program, PC):
        #comprobar que todas las posiciones indicadas tienen sentido en el programa (desbordamientos...)
        #TODO: hacerlo!
        pass
    
    def performComputation(self, program, PC):
        pass
    
    def compute(self, program, PC):
        self.checkValidation(program, PC)
        self.performComputation(program, PC)

        
        
class Addition(Operation):
    def checkValidation(self, program, PC):
        self.checkParameter(program, PC + 1)
        self.checkParameter(program, PC + 2)
        self.checkParameter(program, PC + 3)
        
    def performComputation(self, program, PC):
#         print('Performing Addition')
        program[program[PC + 3]] = program[program[PC + 1]] + program[program[PC + 2]]
        


class Multiply(Operation):
    def performComputation(self, program, PC):
#         print('Performing Multiply')
        program[program[PC + 3]] = program[program[PC + 1]] * program[program[PC + 2]]

    
    
class Quit(Operation):
    def __init__(self):
        self.endComputations = True
    def performComputation(self, program, CP):
#         print('Performing Quit')
        pass

In [ ]:
class IntcodeInterpreter():
    
    PC = 0
    operations = {}
    
    def __init__(self):
        # TODO: Se podría inyectar el juego de instrucciones
        self.operations[1]  = Addition()
        self.operations[2]  = Multiply()
        self.operations[99] = Quit()
    
    def compute(self, program):
        #Analizar el dato que hay en la posición CP
        while True:
            opcode = program[self.PC]
            operation = self.getOperation(opcode)
            
            if operation is None:
                raise Exception('UNKNOWN opcode {} at position {}.\nPROGRAM: {}'.format(opcode, self.PC, program))
            
            if operation.mustEndComputations():
                break
            else:
                operation.compute(program, self.PC)
#                 print('Advancing PC', self.PC, self.PC + 4)
                #TODO: Este 4 podría ser en función de la operación leída
                self.PC += 4
        
        #Devolvemos todo program, dado que no sabemos a priori dónde está el resultado.
        #mmm.. a no ser que nos guardemos dónde ha escrito la última operación computada... :D
        return program

    def getOperation(self, opcode):
        if opcode in self.operations:
            return self.operations[opcode]
        return None

### Tests

In [ ]:
def assertIsEqual(a, b):
    return a == b

#### Test Case 1
1,9,10,3,2,3,11,0,99,30,40,50 becomes 3500,9,10,70, 2,3,11,0, 99, 30,40,50

In [ ]:
program = [1,9,10,3,2,3,11,0,99,30,40,50]
expected = [3500,9,10,70, 2,3,11,0, 99, 30,40,50]

result = IntcodeInterpreter().compute(program)
print(assertIsEqual(result, expected), result, expected)

#### Test Case 2
1,0,0,0,99 becomes 2,0,0,0,99 (1 + 1 = 2).

In [ ]:
program = [1,0,0,0,99]
expected = [2,0,0,0,99]

result = IntcodeInterpreter().compute(program)
print(assertIsEqual(result, expected), result, expected)

#### Test Case 3
2,3,0,3,99 becomes 2,3,0,6,99 (3 * 2 = 6).

In [ ]:
program = [2,3,0,3,99]
expected = [2,3,0,6,99]

result = IntcodeInterpreter().compute(program)
print(assertIsEqual(result, expected), result, expected)

#### Test Case 4
2,4,4,5,99,0 becomes 2,4,4,5,99,9801 (99 * 99 = 9801).

In [ ]:
program = [2,4,4,5,99,0]
expected = [2,4,4,5,99,9801]

result = IntcodeInterpreter().compute(program)
print(assertIsEqual(result, expected), result, expected)

#### Test Case 5
1,1,1,4,99,5,6,0,99 becomes 30,1,1,4,2,5,6,0,99.

In [ ]:
program = [1,1,1,4,99,5,6,0,99]
expected = [30,1,1,4,2,5,6,0,99]

result = IntcodeInterpreter().compute(program)
print(assertIsEqual(result, expected), result, expected)

### Solution

In [ ]:
input_2 = r'C:\Users\rodrigo\Notebooks\aoc2019-input-day2.txt'
# with open(input_2, 'r') as f:
#     data2 = f.read()
with open(input_2, 'r') as f:
    data2 = [int(data) for data in f.read().split(',') if len(data) > 0]    

In [ ]:
data2

before running the program, replace position 1 with the value 12 and replace position 2 with the value 2.

In [ ]:
data2[1] = 12
data2[2] = 2
data2

In [ ]:
IntcodeInterpreter().compute(data2.copy())

## Part 2
Find the input noun and verb that cause the program to produce the output 19690720. What is 100 * noun + verb? (For example, if noun=12 and verb=2, the answer would be 1202.)

In [ ]:
found = False
for noun in range(0, 100):
    if not found:
        for verb in range(0, 100):
            program = data2.copy()
            program[1] = noun
            program[2] = verb
            result = IntcodeInterpreter().compute(program)[0]
            print( noun * 100 + verb, noun, verb, result )
            if result == 19690720:
                found = True
                break
       

# Day 3: Crossed Wires
https://adventofcode.com/2019/day/3

## Part 1

In [ ]:
class Segmento():
    xorigen = yorigen = 0
    xdestino = ydestino = 0
    
    def __init__(self, x0, y0, xd, yd):
        self.xorigen = x0
        self.yorigen = y0
        self.xdestino = xd
        self.ydestino = yd
    
    def isHorizontal(self):
        return self.xorigen != self.xdestino
    
    def getLeftestX(self):
        if self.xorigen <= self.xdestino:
            return self.xorigen
        else:
            return self.xdestino
    
    def getUppestY(self):
        if self.yorigen >= self.ydestino:
            return self.yorigen
        else:
            return self.ydestino
    
    def intersects(self, other):
        if self.isHorizontal():
            if other.isHorizontal():
                #paralelos horizontalmente
                #1. misma altura
                #2. se solapan horizontalmente
                if self.yorigen != other.yorigen:
                    return False
                else:
                    if (self.xorigen >= other.xorigen and self.xorigen <= other.xdestino) or (self.xdestino >= other.xorigen and self.xdestino <= other.xdestino):
                            return True
                    else:
                        return False
            else:
                #Perpendiculares, yo horizontal
                if other.xorigen < self.xorigen or other.xorigen > self.xdestino:
                    return False

    
    def toString(self):
        return '({}, {}) -> ({}, {}) Hor? {} Mas izq {} Mas alto {}'.format(self.xorigen, self.yorigen, self.xdestino, self.ydestino, self.isHorizontal(), self.getLeftestX(), self.getUppestY())
    
def obtainSegments(rawpath):
    path = rawpath.split(',')
    segments = []
    
    xorigen = yorigen = 0
    for step in path:
        xdestino = xorigen
        ydestino = yorigen
        
        print(step, step[:1])
        direction = step[:1]
        distance = int(step[1:])
        if direction == 'U':
            ydestino += distance
        elif direction == 'D':
            ydestino -= distance
        elif direction == 'R':
            xdestino += distance
        elif direction == 'L':
            xdestino -= distance
        else:
            raise Exception('UNKNOWN direction {} from step {}'.format(direction, step))
        
        if distance == 0:
            print('WARNING: Segment of magnitude 0!! Step {}'.format(step))
        
        # Segment in (xorigen, yorigen) -> (xdestino, ydestino)
        print('SEGMENT ({},{}) -> ({}, {})'.format(xorigen, yorigen, xdestino, ydestino))
        
        segments.append(Segmento(xorigen, yorigen, xdestino, ydestino))
        
        xorigen = xdestino
        yorigen = ydestino
    
    return segments

Los hilos se pueden ver como un conjunto de puntos en un espacio discreto bimensional. Obtenemos esos puntos. Los cruces entre ellos será la intersección de los conjuntos de puntos de cada hilo.

In [ ]:
def obtainPoints(rawpath):
    path = rawpath.split(',')
    
    points = []
    xorigen = yorigen = 0
    for step in path:
#         print(step, step[:1])
        direction = step[:1]
        distance = int(step[1:])
        
        if direction == 'U':
            for i in range(distance):
                yorigen += 1
                points.append((xorigen, yorigen))
        elif direction == 'D':
            for i in range(distance):
                yorigen -= 1
                points.append((xorigen, yorigen))
        elif direction == 'R':
            for i in range(distance):
                xorigen += 1
                points.append((xorigen, yorigen))
        elif direction == 'L':
            for i in range(distance):
                xorigen -= 1
                points.append((xorigen, yorigen))
        else:
            raise Exception('UNKNOWN direction {} from step {}'.format(direction, step))

    return points

def getIntersections(wire1, wire2):
    return list(set(wire1).intersection(wire2))

def manhattanPoints(point):
#     print('ENTRANDO', point)
    (x, y) = point
    return abs(x) + abs(y)

def getCloseIntersection(intersections):
    if len(intersections) == 0:
        return None, None
    
    closestIntersection = intersections[:1][0]
    intersections = intersections[1:]
    
    closestDistance = manhattanPoints(closestIntersection)
    for otherIntersection in intersections:
        newDistance = manhattanPoints(otherIntersection)
        if newDistance < closestDistance:
            closestDistance = newDistance
            closestIntersection = otherIntersection
    
    return closestIntersection, closestDistance

In [ ]:
def solveDay3Part1(wire1, wire2):
    puntosHilo1 = obtainPoints(wire1)
    puntosHilo2 = obtainPoints(wire2)
    intersections = getIntersections(puntosHilo1, puntosHilo2)
    return getCloseIntersection(intersections)

### Tests

#### Test 1

* hilo1: R8,U5,L5,D3
* hilo2: U7,R6,D4,L4

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

In [ ]:
hilo1 = 'R8,U5,L5,D3'
hilo2 = 'U7,R6,D4,L4'

solveDay3Part1(hilo1, hilo2)

#### Test 2
* hilo1: R75,D30,R83,U83,L12,D49,R71,U7,L72
* hilo2: U62,R66,U55,R34,D71,R55,D58,R83

distance 159

In [ ]:
hilo1 = 'R75,D30,R83,U83,L12,D49,R71,U7,L72'
hilo2 = 'U62,R66,U55,R34,D71,R55,D58,R83'

solveDay3Part1(hilo1, hilo2)

#### Test 3
* hilo1: R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
* hilo2: U98,R91,D20,R16,D67,R40,U7,R15,U6,R7

distance 135

In [ ]:
hilo1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'
hilo2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'

solveDay3Part1(hilo1, hilo2)

### Solution

In [ ]:
input_3 = r'C:\Users\rodrigo\Notebooks\aoc2019-input-day3.txt'
with open(input_3, 'r') as f:
    realWire1 = f.readline()
    realWire2 = f.readline()
print('WIRE1:', realWire1)
print('WIRE2:', realWire2)

In [ ]:
solveDay3Part1(realWire1, realWire2)

((181, 376), 557)

## Part 2

In [ ]:
def walkWireToPoint(wire, point):
    position = wire.index(point)
    
    if position < 0:
        raise Exception('Intersection {} not found in wire!! {}'.format(point, wire))
    
    #sumamos 1 porque el índice empieza desde 0
    return position + 1

def distanceToIntersection(wire1, wire2, point):
    distance1 = walkWireToPoint(wire1, point)
    distance2 = walkWireToPoint(wire2, point)
    
    return distance1 + distance2

def solveDay3Part2(wire1, wire2):
    puntosHilo1 = obtainPoints(wire1)
    puntosHilo2 = obtainPoints(wire2)
    intersections = getIntersections(puntosHilo1, puntosHilo2)
    
    if len(intersections) == 0:
        return None, None
    
    closestIntersection = intersections[:1][0]
    intersections = intersections[1:]

    closestDistance = distanceToIntersection(puntosHilo1, puntosHilo2, closestIntersection)
    for otherIntersection in intersections:
        newDistance = distanceToIntersection(puntosHilo1, puntosHilo2, otherIntersection)
        if newDistance < closestDistance:
            closestDistance = newDistance
            closestIntersection = otherIntersection
    
    return closestIntersection, closestDistance

### Tests

#### Test 1

* hilo1: R8,U5,L5,D3
* hilo2: U7,R6,D4,L4

The intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

In [ ]:
hilo1 = 'R8,U5,L5,D3'
hilo2 = 'U7,R6,D4,L4'

solveDay3Part2(hilo1, hilo2)

#### Test 2
* hilo1: R75,D30,R83,U83,L12,D49,R71,U7,L72
* hilo2: U62,R66,U55,R34,D71,R55,D58,R83

610 steps

In [ ]:
hilo1 = 'R75,D30,R83,U83,L12,D49,R71,U7,L72'
hilo2 = 'U62,R66,U55,R34,D71,R55,D58,R83'

solveDay3Part2(hilo1, hilo2)

#### Test 3
* hilo1: R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
* hilo2: U98,R91,D20,R16,D67,R40,U7,R15,U6,R7

410 steps

In [ ]:
hilo1 = 'R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51'
hilo2 = 'U98,R91,D20,R16,D67,R40,U7,R15,U6,R7'

solveDay3Part2(hilo1, hilo2)

### Solution

In [ ]:
print('WIRE1:', realWire1)
print('WIRE2:', realWire2)

In [ ]:
solveDay3Part2(realWire1, realWire2)

((1128, 478), 56410)

# Day 4: Secure Container
https://adventofcode.com/2019/day/4


## Part 1

In [ ]:
def validateCodePart1(code):
    #TODO: Revisar para el caso de tamaños impares
    codesize = len(code)
    hayRepetido = False
    for i in range(codesize):
        if i == codesize // 2 :
#             print('vamos por la mitad')
            break
#         print(i, code[i], code[-i-1])
        #1. Los valores del final no pueden ser mayores que los del principio
        if code[-i-1] < code[i]:
#             print('ERROR: desde atrás se decrementa {} > {}'.format(code[-i-1], code[i]))
            return False
        #2. Los valores consecutivos deben ser monónotos crecientes (hacia adelante no miramos la primera posición...)
        if i > 0:
            if code[i] < code[i-1]:
#                 print('ERROR: desde delante no es monótono creciente: {} < {}'.format(code[i], code[i-1]))
                return False
        if code[-i-1] < code[-i-2]:
#                 print('ERROR: desde atrás no es monótono creciente: {} < {}'.format(code[-i-1], code[-i-2]))
                return False
        #3. Miramos a ver si tenemos repetición (por delante y por detrás)
        if code[i] == code[i-1]:
            hayRepetido = True
        if code[-i-1] == code[-i-2]:
            hayRepetido = True

    #En este punto hemos comprobado la condición de monótono creciente del código
    #Así que la validez depende de si hay o no al menos un valor consecutivo igual
    if not hayRepetido:
#         print('ERROR: no pair detected!')
        return False

    return True

### Tests

#### Test 1
111111 meets these criteria (double 11, never decreases).


In [ ]:
code = '111111'
expected = True

result = validateCodePart1(code)
print( result == expected )

#### Test 2
223450 does not meet these criteria (decreasing pair of digits 50).

In [ ]:
code = '223450'
expected = False

result = validateCodePart1(code)
print( result == expected )

#### Test 3
123789 does not meet these criteria (no double).

In [ ]:
code = '123789'
expected = False

result = validateCodePart1(code)
print( result == expected )

### Solution

In [ ]:
raw_input = '172851-675869'
raw_input = raw_input.split('-')

print(raw_input)
lbound = int(raw_input[0])
ubound = int(raw_input[1])

In [ ]:
def solveDay4(validator, lower_bound, upper_bound):
    correctos = 0
    for i in range(lower_bound, upper_bound + 1):
        if validator(str(i)):
            correctos += 1
            print('\t{} es OK!!'.format(i))
        if i % 1000 == 0:
            print(i)

    print('Hay {} códigos correctos.'.format(correctos))

In [ ]:
solveDay4(validateCodePart1, lbound, ubound )

Hay 1660 códigos correctos.

## Part 2

In [ ]:
def validateCodePart2(code):
    codesize = len(code)
    for i in range(codesize):
        if i == codesize // 2 :
#             print('vamos por la mitad')
            break
#         print(i, code[i], code[-i-1])
        #1. Los valores del final no pueden ser mayores que los del principio
        if code[-i-1] < code[i]:
#             print('ERROR: desde atrás se decrementa {} > {}'.format(code[-i-1], code[i]))
            return False
        #2. Los valores consecutivos deben ser monónotos crecientes (hacia adelante no miramos la primera posición...)
        if i > 0:
            if code[i] < code[i-1]:
#                 print('ERROR: desde delante no es monótono creciente: {} < {}'.format(code[i], code[i-1]))
                return False
        if code[-i-1] < code[-i-2]:
#                 print('ERROR: desde atrás no es monótono creciente: {} < {}'.format(code[-i-1], code[-i-2]))
                return False

    #3. Miramos a ver si tenemos repetición hacia adelante
    caracter = None
    ocurrencias = 0
    for i in range(codesize):
#         print('analizando doble {} {}. Ocurrencias {}'.format(i, code[i], ocurrencias))
        if i == 0:
            caracter = code[i]
            ocurrencias = 1
        else:
            if code[i] == code[i-1]:
                #Coincido con la anterior
                ocurrencias += 1
            else:
                #No coincido con la anterior. Si tengo un ocurrencias de 2 es que he cerrado una pareja!
                if ocurrencias == 2:
                    return True
                else:
                    caracter = code[i]
                    ocurrencias = 1

    #Llegar aquí es que no has encontrado pareja durante el recorrido del código, pero aún es posible
    #ser correcto si la pareja está justo al final del código (donde ya no puedes validar ante un cambio
    #porque no hay más símbolos. Eso será si en este punto ocurrencias es exáctamente 2
    return ocurrencias == 2

### Tests

#### Test 1
112233 meets these criteria because the digits never decrease and all repeated digits are exactly two digits long.

In [ ]:
code = '112233'
expected = True

result = validateCodePart2(code)
print( result == expected )

#### Test 2
123444 no longer meets the criteria (the repeated 44 is part of a larger group of 444).

In [ ]:
code = '123444'
expected = False

result = validateCodePart2(code)
print( result == expected )

#### Test 3
111122 meets the criteria (even though 1 is repeated more than twice, it still contains a double 22).

In [ ]:
code = '111122'
expected = True

result = validateCodePart2(code)
print( result == expected )

### Solution

In [ ]:
solveDay4(validateCodePart2, lbound, ubound )

Hay 1135 códigos correctos.